In [ ]:
# !pip install pydantic==1.10.13
# !pip install pydantic==2.6.2

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# Basic Usage

In [2]:
from langchain_zhipu import ChatZhipuAI
llm = ChatZhipuAI()

In [112]:
# 或者你想自己临时指定API_KEY
import os
llm = ChatZhipuAI(api_key=os.getenv('ZHIPUAI_API_KEY'))

## invoke

In [4]:
llm.invoke("你是什么模型？")

AIMessage(content='我是一名人工智能助手，基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-130B 开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。', response_metadata={'id': '8513537775423542980', 'created': 1711533201, 'token_usage': {'prompt_tokens': 9, 'completion_tokens': 49, 'total_tokens': 58}, 'model_name': 'glm-4', 'finish_reason': 'stop'})

## stream

In [5]:
for s in llm.stream("你是什么模型？"):
  print(s.content, end="|", flush=True)

我|是一名|人工智能|助手|，|基于|清华大学| K|EG| 实|验|室|和|智|谱| AI| 公司|于| |202|3| 年|共同|训练|的语言|模型| GL|M|2| 开|发|而成|。|我的|任务是|针对|用户|的问题|和要求|提供|适当的|答复|和支持|。||

In [6]:
async for s in llm.astream("你是什么模型？"):
  print(s.content, end="|", flush=True)

我|是一名|人工智能|助手|，|基于|清华大学| K|EG| 实|验|室|和|智|谱| AI| 公司|于| |202|3| 年|共同|训练|的语言|模型| GL|M|2| 开|发|而成|。|我的|任务是|针对|用户|的问题|和要求|提供|适当的|答复|和支持|。||

## batch

In [18]:
llm.batch([
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？"
])

[AIMessage(content='我使用的模型是清华大学 KEG 实验室和智谱AI共同训练的 GLM 模型，一种基于 Transformer 的通用预训练语言模型。Transformer 模型是一种基于自注意力机制的深度神经网络模型，经常用于处理序列数据。\n\n我可能用到最大的模型是 GLM-130B，具有 1300 亿参数，支持中英双语。我具体使用的模型规模视应用场景可能会有所变化。', response_metadata={'id': '8516977013436319484', 'created': 1711628383, 'token_usage': {'prompt_tokens': 9, 'completion_tokens': 91, 'total_tokens': 100}, 'model_name': 'glm-4', 'finish_reason': 'stop'}),
 AIMessage(content='我是一个人工智能助手模型，基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM2 开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。', response_metadata={'id': '8516978559624613918', 'created': 1711628381, 'token_usage': {'prompt_tokens': 9, 'completion_tokens': 47, 'total_tokens': 56}, 'model_name': 'glm-4', 'finish_reason': 'stop'}),
 AIMessage(content='我是一个基于语言的人工智能模型，名为 ChatGLM。我是由清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-130B 开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', response_metadata={'id': '8516984813097069119', 'created': 1711628382, 'token_usage': {'prompt_tokens': 9, 'completion_tokens': 56, 'total_tokens': 

# Tools-Calling

## function

In [110]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.tools import tool

@tool
def search(query: str) -> str:
    """查询 langchan 资料; args: query 类型为字符串，描述用户的问题."""
    return "langchain_chinese 是一个为中国大模型优化的langchain模块"

llm.bind(tools=[convert_to_openai_tool(search)]).invoke("langchain_chinese是啥？请查询本地资料回答。")


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8513532209145851088', 'function': {'arguments': '{"query":"langchain_chinese是啥？请查询本地资料回答。"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'id': '8513532209145851088', 'created': 1711547168, 'token_usage': {'prompt_tokens': 133, 'completion_tokens': 25, 'total_tokens': 158}, 'model_name': 'glm-4', 'finish_reason': 'tool_calls'})

## web_search

In [9]:
from langchain_zhipu import convert_to_web_search_tool
llm.bind(tools=[convert_to_web_search_tool(search_query="周星驰电影")]).invoke("哪部电影好看？")

AIMessage(content='从您提供的参考信息中，可以看出《银河写手》是一部受到多位电影人如周星驰和姜文高度评价的电影，它在金鸡影展上获得了观众的积极反馈，被认为是一部黑马电影。这部电影由李阔和单丹丹执导，讲述了影视行业的故事，融合了虚幻与现实，是对上世纪90年代北京青春的一种记录。它不仅参考了多部经典影片，还融入了编剧的真实生活经历，试图呈现复古胶片的质感。\n\n因此，如果喜欢深入探讨影视行业、具有文艺气质并且情感真挚的电影，那么《银河写手》可能会是一个不错的选择。\n\n当然，电影的选择也取决于个人的喜好。如果您偏爱喜剧，可能会对周星驰的其他作品如《长江七号》或《美人鱼》感兴趣，这些电影展现了周星驰独特的无厘头风格和他作为导演发掘演员的能力。\n\n总之，选择哪部电影“好看”主要还是根据您个人的兴趣和喜好来定。不同的电影有不同的魅力和价值，建议根据电影的类型、题材、导演和演员阵容等因素来做出选择。', response_metadata={'id': '8513531659390035186', 'created': 1711532324, 'token_usage': {'prompt_tokens': 616, 'completion_tokens': 220, 'total_tokens': 836}, 'model_name': 'glm-4', 'finish_reason': 'stop'})

## retrieval

In [107]:
from langchain_zhipu import convert_to_retrieval_tool
llm.bind(tools=[convert_to_retrieval_tool(knowledge_id="1772979648448397312")]).invoke("你知道马冬梅住哪里吗？")

AIMessage(content='马冬梅住在楼上322房间。', response_metadata={'id': '8513532209145848833', 'created': 1711546897, 'token_usage': {'prompt_tokens': 82, 'completion_tokens': 11, 'total_tokens': 93}, 'model_name': 'glm-4', 'finish_reason': 'stop'})

In [10]:
llm.bind(tools=[convert_to_openai_tool(search)])

RunnableBinding(bound=ChatZhipuAI(client=<langchain_zhipu._client.ZhipuAI object at 0x10c437220>), kwargs={'tools': [{'type': 'function', 'function': {'name': 'search', 'description': 'search(query: str) -> str - 查询 langchan 资料; args: query 类型为字符串，描述用户的问题.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}}}]})

# 使用官方接口

In [98]:
from langchain_zhipu import ZhipuAI

client = ZhipuAI()

tools = [
        {
            "type": "function",
            "function": {
                "name": "search",
                "description": "search(query: str) -> str - 查询 langchan 资料; args: query 类型为字符串，描述用户的问题.'",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string"
                        },
                    },
                    "required": ["query"]
                }
            }
        }
    ]

In [100]:
client.chat.completions.create(
    model="glm-4",
    messages = [
        {
            "role": "user",
            "content": "langchain是什么？请帮我查询"
        }
    ],
    tools = tools,
    tool_choice="auto",
)

Completion(model='glm-4', created=1711538632, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8513538359539114557', function=Function(arguments='{"query":"langchain是什么？"}', name='search'), type='function')]))], request_id='8513538359539114557', id='8513538359539114557', usage=CompletionUsage(prompt_tokens=128, completion_tokens=16, total_tokens=144))

In [99]:
client.chat.completions.create(
    model="glm-4",
    messages = [
        {
            "role": "user",
            "content": "langchain是什么？"
        }
    ],
    tools = tools,
    tool_choice="auto",
)

APIInternalError: Error code: 500, with error text {"error":{"code":"500","message":"Internal Error"}}

# 智能体

In [9]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

tools = [search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for chunk in agent_executor.stream({"input": "langchain_chinese是什么？"}):
    print(chunk)




> Entering new AgentExecutor chain...
{'actions': [OpenAIToolAgentAction(tool='search', tool_input={'query': 'langchain_chinese是什么'}, log="\nInvoking: `search` with `{'query': 'langchain_chinese是什么'}`\nresponded: 我需要使用搜索引擎来获取相关信息。<|assistant|>\n\n", message_log=[AIMessageChunk(content='我需要使用搜索引擎来获取相关信息。<|assistant|>', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8513530731677143587', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_callstool_calls'})], tool_call_id='call_8513530731677143587')], 'messages': [AIMessageChunk(content='我需要使用搜索引擎来获取相关信息。<|assistant|>', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8513530731677143587', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_callstool_calls'})]}

Invoking: `search` with `{'query': 'langchain_chinese是什么'}`
responde

In [33]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

# GLM-4V

In [13]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate

llm4v = ChatZhipuAI(model="glm-4v")

prompt = ChatPromptTemplate.from_messages([
    ("human", [
          {
            "type": "text",
            "text": "图里有什么"
          },
          {
            "type": "image_url",
            "image_url": {
                "url" : "https://img1.baidu.com/it/u=1369931113,3388870256&fm=253&app=138&size=w931&n=0&f=JPEG&fmt=auto?sec=1703696400&t=f3028c7a1dca43a080aeb8239f09cc2f"
            }
          }
        ]),
])

(prompt|llm4v).invoke({})

AIMessage(content='图中有一片蓝天白云，海面，岩石，以及右下角的树木。具体地，在图片右边有深绿色的树木，树的右边是灰黑色的岩石滩，延伸到海中。海的另一侧有远方的陆地，陆地上似乎有一些建筑。天空是蓝色的，上面飘着一些白色的云朵。')

# get_token_ids

In [17]:
from langchain_zhipu import ChatZhipuAI

llm = ChatZhipuAI(allowed_special=set())
llm.get_token_ids('<|user|>')

[27, 91, 882, 91, 29]

In [5]:
llm.get_num_tokens(text="你好")

2